In [14]:
minmaxInterval = 2000

## open port

In [15]:
!echo 'quzinee1!'| sudo -S chmod a+rw /dev/ttyUSB0

[sudo] password for terry: 

## BASIC USAGE OF XL320 CLASS

In [16]:
import rospy
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint

rospy.init_node("Experiment2")


class Soro(object):
    def __init__(self):
        self.transform_data_pub = rospy.Publisher('/dynamixel/control', JointTrajectory, queue_size=10)

    def actuate(self, data:JointTrajectory):
        self.transform_data_pub.publish(data)


soro = Soro()
rate = rospy.Rate(1) # 10 Hz

In [17]:
import sys
sys.path.append(".")
""" FOR DYNAMIXEL """
from model.class_xc330 import xc330
""" FOR MODERN DRIVER """
import roslib; roslib.load_manifest('ur_driver')
import rospy
import actionlib
from control_msgs.msg import *
from trajectory_msgs.msg import *
from sensor_msgs.msg import JointState

from class_vicon_marker import VICON
from class_vicon_base import ViconBase
import copy
import numpy as np

from numpy import pi as PI
DEG90 = PI/2

class UR(object):
    def __init__(self,):
        self.JOINT_NAMES = ['shoulder_pan_joint', 'shoulder_lift_joint', 'elbow_joint',
                            'wrist_1_joint', 'wrist_2_joint', 'wrist_3_joint']
        self.client      = None
        
   
    def move_arm_speed(self, traj:JointTrajectory, speed_limit):
        try: 
            g = FollowJointTrajectoryGoal()
            g.trajectory = copy.deepcopy(traj)
            g.trajectory.joint_names = self.JOINT_NAMES
            joint_states = rospy.wait_for_message("joint_states", JointState)
            joints_pos   = joint_states.position

            init_point = JointTrajectoryPoint()
            init_point.positions = joints_pos
            init_point.time_from_start = rospy.Duration.from_sec(0)
            init_point.velocities = [0 for _ in range(6)]
            g.trajectory.points.insert(0, init_point)

            q_list = []
            time_list = []
            for point in g.trajectory.points:
                q_list.append(point.positions)
                time_list.append(point.time_from_start.to_sec())

            print(q_list)

            for i in range(len(q_list)-1):
                q_before = np.array(q_list[i])
                q_after  = np.array(q_list[i+1])
                print(q_after, q_before)
                time_before = time_list[i]
                time_after = time_list[i+1]

                diff_q = q_after - q_before
                diff_time = time_after - time_before

                speed = np.linalg.norm(diff_q)/diff_time
                print(speed)
                assert speed <= speed_limit
            
            

            self.client.send_goal(g)
            # self.client.wait_for_result()
        except KeyboardInterrupt:
            self.client.cancel_goal()
            raise
        except:
            raise  

    def execute_arm_speed(self, traj, speed_limit):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_speed(traj, speed_limit)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      

    def execute_arm_time(self, joints, movetime):
        try:
            self.client = actionlib.SimpleActionClient('follow_joint_trajectory', FollowJointTrajectoryAction)
            print("Waiting for server...")
            self.client.wait_for_server()
            print("Connected to server")
            """ Initialize """
            self.move_arm_time(joints, movetime)
            print("Finish plan")

        except KeyboardInterrupt:
            rospy.signal_shutdown("KeyboardInterrupt")
            raise      


## INSTANTIATE Robot 

In [18]:
robot = UR()
init_joint =[0,-DEG90,DEG90,-DEG90,-DEG90, DEG90]

In [19]:
q =[0,-DEG90,DEG90,-DEG90,-DEG90, DEG90 * 0]

q_traj = JointTrajectory()

point = JointTrajectoryPoint()
point.positions = q
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(5)

q_traj.points.append(point)

robot.execute_arm_speed(q_traj, speed_limit=0.5)
# traj

Waiting for server...
Connected to server
[(0.0004184246063232422, -1.5709864101805628, 1.5707595984088343, -1.5705300134471436, -1.5707858244525355, 0.0004286766052246094), [0, -1.5707963267948966, 1.5707963267948966, -1.5707963267948966, -1.5707963267948966, 0.0]]
[ 0.         -1.57079633  1.57079633 -1.57079633 -1.57079633  0.        ] [ 4.18424606e-04 -1.57098641e+00  1.57075960e+00 -1.57053001e+00
 -1.57078582e+00  4.28676605e-04]
0.00013672696139082583
Finish plan


In [20]:
motor_control = [0,0]

motor_traj = JointTrajectory()
point = JointTrajectoryPoint()
point.positions = motor_control
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(2)

motor_traj.points.append(point)
soro.actuate(motor_traj)

In [21]:
runname = "R"
motor_array = np.load(f"planned_traj/{runname}/motor_array.npy")
qs_array = np.load(f"planned_traj/{runname}/qs_array.npy")

temp = motor_array.copy()

temp[:,0] = motor_array[:,1]
temp[:,1] = -motor_array[:,0]
motor_array = temp

motor_array.shape, qs_array.shape

((140, 2), (140, 6))

In [22]:
q_traj = JointTrajectory()
motor_traj = JointTrajectory()

q_before = np.array(rospy.wait_for_message("joint_states", JointState).position)

unit_time = 1
track_time = 0

for i,(qs, motor_control) in enumerate(zip(qs_array, motor_array)):
    if i == 0:
        unit_time = 5
    else:
        unit_time = 0.5
    
    delta_q = np.linalg.norm(qs - q_before)

    unit_time = max(delta_q/0.2, unit_time)
    
    track_time = track_time + unit_time
    point = JointTrajectoryPoint()
    point.positions = qs
    point.velocities = [0 for _ in range(6)]
    point.time_from_start = rospy.Duration.from_sec(track_time)
    q_traj.points.append(point)
    
    point = JointTrajectoryPoint()
    point.positions = motor_control
    point.velocities = [0 for _ in range(2)]
    point.time_from_start = rospy.Duration.from_sec(track_time)
    motor_traj.points.append(point)

    q_before = qs

In [23]:
# robot.execute_arm_speed(q_traj, speed_limit=0.5)
# soro.actuate(motor_traj)

In [24]:
# robot.client.wait_for_result()

# Final DEMO

In [25]:
q_ink_up   = np.array([-52.8, -80.8+2, 110.9-2, -121.3, -89.9, -52.7], dtype=np.float32) / 180 * PI
q_ink_down = np.array([-52.8, -75.8+2, 115.0-2, -130.3, -89.9, -52.7], dtype=np.float32) / 180 * PI

m_ink_up   = np.array([0,-800], dtype=np.float32)
m_ink_down = np.array([0,800], dtype=np.float32)

In [26]:
q_ink_traj = JointTrajectory()
motor_ink_traj = JointTrajectory()

point = JointTrajectoryPoint()
point.positions = q_ink_up
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(5)
q_ink_traj.points.append(point)

point = JointTrajectoryPoint()
point.positions = q_ink_down
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(7)
q_ink_traj.points.append(point)

point = JointTrajectoryPoint()
point.positions = q_ink_down
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(9)
q_ink_traj.points.append(point)

point = JointTrajectoryPoint()
point.positions = q_ink_up
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(11)
q_ink_traj.points.append(point)


point = JointTrajectoryPoint()
point.positions = m_ink_up
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(5)
motor_ink_traj.points.append(point)

point = JointTrajectoryPoint()
point.positions = m_ink_up
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(7)
motor_ink_traj.points.append(point)

point = JointTrajectoryPoint()
point.positions = m_ink_down
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(9)
motor_ink_traj.points.append(point)

point = JointTrajectoryPoint()
point.positions = m_ink_down
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(11)
motor_ink_traj.points.append(point)


# robot.execute_arm_speed(q_ink_traj, speed_limit=0.5)
# soro.actuate(motor_ink_traj)


In [27]:
import time

for runname in ["M","I","L","A","B"]:
# for runname in ["M", 'B']:
    robot.execute_arm_speed(q_ink_traj, speed_limit=0.5)
    soro.actuate(motor_ink_traj)
    robot.client.wait_for_result()
    time.sleep(0.5)


    motor_array = np.load(f"planned_traj/{runname}/motor_array.npy")
    qs_array = np.load(f"planned_traj/{runname}/qs_array.npy")

    temp = motor_array.copy()

    temp[:,0] = motor_array[:,1]
    temp[:,1] = -motor_array[:,0]
    motor_array = temp

    q_traj = JointTrajectory()
    motor_traj = JointTrajectory()

    q_before = np.array(rospy.wait_for_message("joint_states", JointState).position)

    unit_time = 1
    track_time = 0



    for i,(qs, motor_control) in enumerate(zip(qs_array, motor_array)):
        if i == 0:
            unit_time = 5
        else:
            unit_time = 0.3
        
        delta_q = np.linalg.norm(qs - q_before)

        unit_time = max(delta_q/0.2, unit_time)
        
        track_time = track_time + unit_time
        point = JointTrajectoryPoint()
        point.positions = qs
        point.velocities = [0 for _ in range(6)]
        point.time_from_start = rospy.Duration.from_sec(track_time)
        q_traj.points.append(point)
        
        point = JointTrajectoryPoint()
        point.positions = motor_control
        point.velocities = [0 for _ in range(2)]
        point.time_from_start = rospy.Duration.from_sec(track_time)
        motor_traj.points.append(point)

        q_before = qs
        
    robot.execute_arm_speed(q_traj, speed_limit=0.5)
    soro.actuate(motor_traj)

    robot.client.wait_for_result()
    time.sleep(0.5)



q =[0,-DEG90,DEG90,-DEG90,-DEG90, DEG90 * 0]

q_traj = JointTrajectory()

point = JointTrajectoryPoint()
point.positions = q
point.velocities = [0 for _ in range(6)]
point.time_from_start = rospy.Duration.from_sec(5)

q_traj.points.append(point)


motor_control = [0,0]

motor_traj = JointTrajectory()
point = JointTrajectoryPoint()
point.positions = motor_control
point.velocities = [0 for _ in range(2)]
point.time_from_start = rospy.Duration.from_sec(2)

motor_traj.points.append(point)

robot.execute_arm_speed(q_traj, speed_limit=0.5)
soro.actuate(motor_traj)

robot.client.wait_for_result()
time.sleep(0.5)


Waiting for server...
Connected to server
[(0.00020271900575608015, -1.5709148846068324, 1.5707595984088343, -1.5706857426087772, -1.5707739035235804, 0.00016492605209350586), array([-0.9215338 , -1.3753195 ,  1.9006636 , -2.1170845 , -1.569051  ,
       -0.91978854], dtype=float32), array([-0.9215338 , -1.2880532 ,  1.9722221 , -2.2741642 , -1.569051  ,
       -0.91978854], dtype=float32), array([-0.9215338 , -1.2880532 ,  1.9722221 , -2.2741642 , -1.569051  ,
       -0.91978854], dtype=float32), array([-0.9215338 , -1.3753195 ,  1.9006636 , -2.1170845 , -1.569051  ,
       -0.91978854], dtype=float32)]
[-0.9215338  -1.3753195   1.9006636  -2.1170845  -1.569051   -0.91978854] [ 2.02719006e-04 -1.57091488e+00  1.57075960e+00 -1.57068574e+00
 -1.57077390e+00  1.64926052e-04]
0.29268149425571643
[-0.9215338  -1.2880532   1.9722221  -2.2741642  -1.569051   -0.91978854] [-0.9215338  -1.3753195   1.9006636  -2.1170845  -1.569051   -0.91978854]
0.09670840203762054
[-0.9215338  -1.2880532   1